<a href="https://colab.research.google.com/github/ElenaShargina/nnetworks/blob/main/%D0%9C%D0%BE%D0%B4%D0%B5%D0%BB%D1%8C_%D0%B8%D1%81%D0%BA%D1%83%D1%81%D1%81%D1%82%D0%B2%D0%B5%D0%BD%D0%BD%D0%BE%D0%B3%D0%BE_%D0%BD%D0%B5%D0%B9%D1%80%D0%BE%D0%BD%D0%B0_%D0%B8_%D0%B3%D1%80%D0%B0%D0%B4%D0%B8%D0%B5%D0%BD%D1%82%D0%BD%D1%8B%D0%B9_%D1%81%D0%BF%D1%83%D1%81%D0%BA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Реализована работа базовой модели искусственного нейрона Мак-Каллока – Питтса.

Функция активация - сигмоидальная.
Корректировка весов происходит с помощью стохастического градиентного спуска.

# Подготовка данных

In [91]:
# Импортируем необходимые пакеты
import numpy as np

In [92]:
# количество элементов в обучающей выборке
m = 17
# величина шага
h = 2
# количество эпох без улучшения точности, после которого остановим поиск
patience = 10

# точность
eps = 0.000001

# обучающая выборка
dataX = [
    [0.1, 0.2, 0.3, 0.4],
    [0.2, 0.3, 0.4, 0.5],
    [0.3, 0.4, 0.5, 0.6],
    [0.4, 0.5, 0.6, 0.7],
    [0.5, 0.6, 0.7, 0.8],
    [0.6, 0.7, 0.8, 0.9],
    [0.7, 0.8, 0.9, 0],
    [0.8, 0.9, 0, -0.1],
    [0.9, 0, -0.1, -0.2],
    [0, -0.1, -0.2, -0.3],
    [-0.1, -0.2, -0.3, -0.4],
    [-0.2, -0.3, -0.4, -0.5],
    [-0.3, -0.4, -0.5, -0.6],
    [-0.4, -0.5, -0.6, -0.7],
    [-0.5, -0.6, -0.7, -0.8],
    [-0.6, -0.7, -0.8, -0.9],
    [-0.7, -0.8, -0.9, 0]
]
# правильные ответы к обучающей выборке
dataY = [1, 1.2, 1.3, 1.4, 1.5, 1.6, 1.7, 1.8, 1.9, 0, -1, -1.2, -1.3, -1.4, -1.5, -1.6, -1.7]


Задаем первичные веса - каике-то маленькие значения.

In [93]:
w1 = 0.1
w2 = 0.2
w3 = 0.3
w4 = 0.4

# Задание функций

Функция активации - сигмоидная

$\phi(x) = \frac{1}{1+e^{-x}}$

In [94]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

Производная от сигмоидной функции

$ \phi^\prime(x) = \frac{e^{x}}{(1+e^{x})^2}$

In [95]:
def diffsig(x): 
    return np.exp(x)/(1+np.exp(x))**2

Функция потерь

$\sum_{i=1}^m{(y-\phi(\sum_{i=1}^n{w_ix_i}))^2}$

In [96]:
def loss_func(w1, w2, w3, w4, dataY, dataX, m=17):
    E = 0
    for i in range(m):
        E += (dataY[i] - sigmoid(w1 * dataX[i][0] + w2 * dataX[i][1] + w3 * dataX[i][2] + w4 * dataX[i][3]))**2 
    return E

Производная функции потерь по j-му весу

$\frac{\partial \phi(x)}{\partial x_j} = {2(y-\phi({\sum_{i=1}^n}{w_ix_i}))\phi\prime({\sum_{i=1}^n}{w_ix_i})x_j}$

In [97]:
def diff_loss_func(j, w1, w2, w3, w4, Y, X):
    diffE = 2 * (Y - sigmoid(w1 * X[0] + w2 * X[1] + w3 * X[2] + w4 * X[3])
                 ) * diffsig(w1 * X[0] + w2 * X[1] + w3 * X[2] + w4 * X[3]
                             ) * X[j]
    return diffE

# Работа алгоритма

Стохастический градиентный спуск - после анализа каждого элемента выборки будем менять веса модели по формуле
$w_i=w_i+h\frac{\partial \phi(x)}{\partial x_j}$


In [98]:
# счетчик итераций без улучшений, 
# заканчиваем цикл, если он достигнет параметра patience
failure_stop = 0
# порядок просмотра элементов выборки,
# будем перемешивать его каждую эпоху
index = np.arange(m)

for k in range(4000):
    # Перемешиваем обучающую выборку
    np.random.shuffle(index)
    # loss_old - значение функции потерь на предыдущей итерации, 
    # на первой итерации не существует
    if k>0:
      loss_old = loss_new

    # Изменяем веса после анализа каждого элемента выборки
    for i in index:
        w1 = w1 + h * diff_loss_func(0, w1, w2, w3, w4, dataY[i], dataX[i])
        w2 = w2 + h * diff_loss_func(1, w1, w2, w3, w4, dataY[i], dataX[i])
        w3 = w3 + h * diff_loss_func(2, w1, w2, w3, w4, dataY[i], dataX[i])
        w4 = w4 + h * diff_loss_func(3, w1, w2, w3, w4, dataY[i], dataX[i])
    
    # высчитываем функцию потерь на новых весах
    loss_new = loss_func(w1, w2, w3, w4, dataY, dataX)

    # если итерация не первая, то проверяем, улучшилась ли точность
    if k>0 and abs(loss_new - loss_old) < eps:
        failure_stop += 1
    else:
        failure_stop = 0

    # если точность не улучшилась уже заданное количество итераций, то останавливаем цикл    
    if failure_stop == patience:
        print(f'Количество итераций без улучшений: {failure_stop}, останавливаем работу алгоритма')
        break

# выводим результирующие данные
print(f'Получившиеся значения весов: {w1:.6f},{w2:.6f},{w3:.6f},{w4:.6f}')
print(f'Количество итераций алгоритма: {k}')
print(f'Значение функции потерь на обучающей выборке: {loss_new:.6f}')

Количество итераций без улучшений: 10, останавливаем работу алгоритма
Получившиеся значения весов: 11.855310,5.922691,7.014721,8.194374
Количество итераций алгоритма: 1463
Значение функции потерь на обучающей выборке: 16.631474
